# **Projeto do estimador**

Considere um sistema
$$
\begin{align}
    \dot{\mathbf{x}} & = \mathbf{Ax+B}u\\
    y &= \mathbf{Cx}
\end{align}
$$
e o observador
$$
\begin{align}
    \dot{\mathbf{\hat{x}}} & = \mathbf{A\hat{x}+B}u+\mathbf{L}(y-\mathbf{C\hat{x})}\\
    &= \mathbf{A\hat{x}+B}u+\mathbf{LC}(\mathbf{x-\hat{x})}
\end{align}
$$

Subtraindo as duas equações de estado, obtemos:
$$
\begin{align}
    \dot{\mathbf{x}}-\dot{\mathbf{\hat{x}}} & = \mathbf{(A-LC)(x-\hat{x}})
\end{align}
$$

O sinal $\mathbf{e} = \mathbf{x}-\mathbf{\hat{x}}$ representa o erro de estimação entre os estados. Então: 
$$
\begin{align*}
    \dot{\mathbf{e}} &= \mathbf{(A-LC)e}
\end{align*}
$$

Esse é um sistema autônomo (sem entrada), que só depende das condições iniciais. 

Se as condições iniciais fossem iguais, $\mathbf{x}(0)=\mathbf{\hat{x}}(0)$, então $\mathbf{e}(0)=\mathbf{0}$, e o erro $\mathbf{e}(t)$ permaneceria nulo indefinidamente. 

Na prática, porém, não conhecemos as condições iniciais do sistema e assim $\mathbf{e}(0)\neq \mathbf{0}$.

Apesar disso, podemos fazer $\mathbf{e}(t)$ tender para zero depois de um tempo. Para isso, basta que a matriz $\mathbf{(A-LC)}$ seja estável, isto é, tenha todos os autovalores no SPE. 


Esse é um problema semelhante ao da regulação de estados. Nele, queremos escolher $\mathbf{K}$ (matriz linha), tal que $\mathbf{A-BK}$ seja estável.

No observador queremos escolher $\mathbf{L}$ (matriz coluna), tal que $\mathbf{A-LC}$ seja estável. Note que a diferença é que:
- No observador usamos $\mathbf{C}$ (linha) ao invés de $\mathbf{B}$ (coluna)
- No observador usamos $\mathbf{L}$ (coluna) ao invés de $\mathbf{K}$ (linha)
- A ordem de $\mathbf{L}$ está trocada na equação (à esquerda ao invés de direita)

Nestas condições, resolver o problema do observador é o mesmo que resolver o problema do regulador com as matrizes $\mathbf{A}^T$ e $\mathbf{C}^T$, para encontrar o vetor de ganhos $\mathbf{L}^T$. 

Isso é verdade porque se transpusermos a matriz $\mathbf{A-LC}$:
$$
\begin{align}
    (\mathbf{A-LC})^T = \mathbf{A}^T-\mathbf{C}^T\mathbf{L}^T
\end{align}
$$

Neste formato, o projeto fica **idêntico** ao do regulador, porque:
- $\mathbf{A}^T$ é quadrada (tal como $\mathbf{A}$ no regulador)
- $\mathbf{C}^T$ é coluna (tal como $\mathbf{B}$ no regulador)
- $\mathbf{L}^T$ é linha (tal como $\mathbf{K}$ no regulador)

Assim, podemos usar as mesmas estratégias e funções Python que usamos no regulador para projetar o estimador. 

## Exemplo:

<p align="center">
<img src="Ex7.25.svg" width="80%">
</p>

Como antes, vamos resolver primeiro usando simbólico.

**Como estamos resolvendo "manualmente", não precisamos usar o artífio de transposição**

In [1]:
import numpy as np
import sympy as sp
from escrever import *

esc = escrever()

In [3]:
s = sp.symbols('s')
w0 = sp.symbols('\omega_0')
l1,l2 = sp.symbols(['l_1','l_2'])

In [4]:
# Matrizes do sistema
A = sp.Matrix([[0,1],[-w0**2,0]])
C = sp.Matrix([1,0]).T
L = sp.Matrix([[l1],[l2]])
display(Math(r'\mathbf{A = }' + sp.latex(A)))
display(Math(r'\mathbf{C = }' + sp.latex(C)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
#Poliômio do observador
display(Math(r's\mathbf{I-A+LC = }'))
eq1 = '= s' + sp.latex(sp.eye(2))  # sI
eq1 += '-' + sp.latex(A) + '+' + sp.latex(L) + sp.latex(C) +'='
eq2 = '= ' + sp.latex(s*sp.eye(2)) + '-'  + sp.latex(A) + '+' + sp.latex(L@C) + ' = '
eq3 = '= ' + sp.latex(s*sp.eye(2)-A+L@C)
display(Math(eq1))
display(Math(eq2))
display(Math(eq3))

# Polinômio do observador (A-LC)
p_obsv = sp.det(s*sp.eye(2)-A+L@C)
display(Math(r'|s\mathbf{I-A+LC}| = ' + sp.latex(p_obsv)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
# Polinômio desejado
p_desejado = sp.expand((s+10*w0)**2)
display(Math(r'\alpha_o(s) = ' + sp.latex((s+10*w0)**2) + '= ' + sp.latex(p_desejado)))

<IPython.core.display.Math object>

In [17]:
# Igualando termo a termo
display(Math(r'|s\mathbf{I-A+LC}| = \alpha_o(s)'))
eq = [sp.Eq(p_obsv.coeff(s,n),p_desejado.coeff(s,n)) for n in (0,1)]
display(Math(sp.latex(p_obsv) + '=' + sp.latex(p_desejado)))
display(Math(sp.latex(eq[0])))
display(Math(sp.latex(eq[1])))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [21]:
# Resolve o sistema
sol = sp.solve(eq,L)
display(Math(sp.latex(l1) + '= ' + sp.latex(sol[l1])))
display(Math(sp.latex(l2) + '= ' + sp.latex(sol[l2])))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**Solução numérica**

Se adotarmos $\omega_0=1$, podemos resolver o mesmo problema numericamente usando a função de posicionamento de polos *acker()* e as matrizes transpostas. 

In [22]:
import control as ct

In [27]:
# Declara as matrizes
w0=1
A = np.array([[0,1],[-w0**2,0]])
C = np.array([[1,0]])
display(Math(r'\mathbf{A} = ' +  esc.mat(A)))
display(Math(r'\mathbf{C} = ' +  esc.mat(C)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [25]:
# Posiciona os polos
L = ct.acker(A=A.T,B=C.T,poles=[-10*w0,-10*w0]).T
display(Math(r'\mathbf{L = }' + esc.mat(L)))

<IPython.core.display.Math object>